# Домашнее задание №4 - CNN Свертки

1. Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) сверточной сетью (самописной)

2. Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50

3. Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50 с аугментацией (самописной, с использованием Pytorch встроенных методов)

In [54]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm
from random import randint, random

from sklearn.model_selection import train_test_split

In [2]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True)

Files already downloaded and verified


In [86]:
class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None, augment_transforms=None, augment_p=0.3):
        self._base_dataset = init_dataset
        self.transform = transform
        self.augment_transforms = augment_transforms
        self.augment_p = augment_p

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]

        if self.augment_transforms and random() < self.augment_p:
            at = self.augment_transforms[randint(0, len(self.augment_transforms)-1)]
            if at:
                img = at(img)

        if self.transform is not None:
            img = self.transform(img)
        
        return img, self._base_dataset[idx][1]

    
def get_datasets(ds, image_size=32, augment_transforms=None, augment_p=0.3, test_size=0.05):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


    train_transform = transforms.Compose([transforms.Scale(int(image_size*1.2)),
                                        transforms.RandomCrop(image_size, padding=4),
                                        transforms.ToTensor(),
                                        normalize])
    valid_transform = transforms.Compose([transforms.Scale(image_size),
                                        transforms.ToTensor(),
                                        normalize])

    train_dataset, valid_dataset = train_test_split(ds, test_size=0.05, random_state=13)

    train_dataset = MyOwnCifar(train_dataset, train_transform, augment_transforms, augment_p)
    valid_dataset = MyOwnCifar(valid_dataset, valid_transform)
    return train_dataset, valid_dataset

train_dataset, valid_dataset = get_datasets(dataset)

In [4]:
def evaluate(net, ds):
    dl = torch.utils.data.DataLoader(ds, batch_size=128, shuffle=False)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.eval()
    total_loss = 0
    count = 0
    matches = 0
    for X, y in dl:
        count += len(y)
        preds = net(X)
        total_loss += float(loss(preds, y)) # очень важно преобразовать во float здесь, иначе утекает память !!!
        pred_classes = torch.argmax(preds, dim=1)
        matches += int(sum(pred_classes == y))
    return float(total_loss/count), float(matches/count), count
 
def train_and_test(net, ds_train, ds_test, optimizer_class=torch.optim.Adam, n_epochs=10, lr=0.01, report_on=-1, batch_size=1024):
    
    print('========================================================')
    print(' Обучаем сеть: ', net)
    
    params = [ p for p in net.parameters() if p.requires_grad]
    print('Число обучаемых параметров', len(params))
    optimizer = optimizer_class(params, lr=0.01)
    print(' Оптимизатор: ', optimizer)

    def print_results():
        for title, ds in [
#                ('Тренировочный', ds_train), # слишком долго для resnet 
                ('Валидационный', ds_test)
            ]:
            r = evaluate(net, ds)
            print(f"    {title} набор: кросс-энтропия: {r[0]:.2f}, доля совпадений: {r[1]*100:.1f}%")
    print(' До обучения')
    print_results()
    
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()    

    net.train()
    report_on = max(len(dl_train) if report_on == -1 else report_on, 1)
    for epoch in range(n_epochs):
        print('--------- Эпоха ', epoch, '/', n_epochs, ' ------------------')
        running_loss = 0.0
        for i, data in enumerate(tqdm(dl_train)):
            inputs, labels = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if (i+1) % report_on == 0:    # печатаем каждые report_on mini-batches
                print('Выучено батчей : %5d; loss: %.3f' % (i + 1, running_loss / report_on))
                running_loss = 0.0
                
    print(' Обучение закончено!')
    print(' После обучения')
    print_results()


In [17]:
class Net(nn.Module):

    def __init__(self, image_size, output_classes=100):
        super(Net, self).__init__()
        self.dp_three = nn.Dropout(0.2)
        self.dp_four = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, 3)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 60, 3)
        self.bn_three = torch.nn.BatchNorm2d(60)
        self.conv_three = torch.nn.Conv2d(60, 120, 3)
        self.bn_four = torch.nn.BatchNorm2d(120)
        n = (((image_size - 2) // 2 - 2) // 2 - 2) // 2
        n = n * n * 120        
        self.fc1 = torch.nn.Linear(n, 200)
        self.fc2 = torch.nn.Linear(200, 60)
        self.out = torch.nn.Linear(60, output_classes)
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_three(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_four(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.out(x)

net = Net(train_dataset[0][0].shape[1], 100)
print(net)

Net(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=100, bias=True)
)


In [18]:
for i in range(5):
    train_and_test(net, train_dataset, valid_dataset, n_epochs=5) #5 по 5 эпох с выводом промежуточного результата

 Обучаем сеть:  Net(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=100, bias=True)
)
Число обучаемых параметров 20
 Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:37<00:00,  1.24it/s]


Выучено батчей :    47; loss: 4.080
--------- Эпоха  1 / 5  ------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:38<00:00,  1.21it/s]


Выучено батчей :    47; loss: 3.670
--------- Эпоха  2 / 5  ------------------


...

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:38<00:00,  1.22it/s]


Выучено батчей :    47; loss: 2.559
--------- Эпоха  4 / 5  ------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:38<00:00,  1.21it/s]

Выучено батчей :    47; loss: 2.548
 Обучение закончено!
 После обучения


    Валидационный набор: кросс-энтропия: 2.81, доля совпадений: 31.6%


In [5]:
from torchvision import models

resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = False

resnet50.fc = nn.Linear(2048, 100)

In [42]:
train_dataset, valid_dataset = get_datasets(dataset, image_size=224)

В следующей строке проверка на valid_dataset неправильная, его надоувеличить до 224x224, но pytorch очень медленно отрабатывает, поэтому я не перезапускала строку. Но проверка на valid_dataset сделана в последующей строке.

In [6]:
train_and_test(resnet50, train_dataset, valid_dataset, n_epochs=5)

 Обучаем сеть:  ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1,

100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:01:49<00:00, 78.92s/it]


Выучено батчей :    47; loss: 8.819
--------- Эпоха  1 / 5  ------------------


100%|███████████████████████████████████████████████████████████████████████████████| 47/47 [4:59:55<00:00, 382.88s/it]


Выучено батчей :    47; loss: 1.894
--------- Эпоха  2 / 5  ------------------


...

100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:00:23<00:00, 77.09s/it]


Выучено батчей :    47; loss: 1.556
--------- Эпоха  4 / 5  ------------------


100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:00:27<00:00, 77.18s/it]

Выучено батчей :    47; loss: 1.505
 Обучение закончено!
 После обучения


    Валидационный набор: кросс-энтропия: 18.88, доля совпадений: 4.7%


In [29]:
r = evaluate(resnet50, valid_dataset)
print(f"Валидационный набор: кросс-энтропия: {r[0]:.2f}, доля совпадений: {r[1]*100:.1f}%")

Валидационный набор: кросс-энтропия: 1.61, доля совпадений: 57.2%


In [87]:
train_dataset, valid_dataset = get_datasets(dataset, image_size=224, augment_transforms=[
        transforms.RandomRotation(180),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.RandomVerticalFlip(p=1.0),
        transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
    ], augment_p = 0.5)


Продолжу обучение на той же сети с аугментацией, так как слишком медленно идет процесс

In [89]:
train_and_test(resnet50, train_dataset, valid_dataset, n_epochs=3)

 Обучаем сеть:  ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1,

100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:02:37<00:00, 79.95s/it]


Выучено батчей :    47; loss: 5.087
--------- Эпоха  1 / 3  ------------------


100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:01:40<00:00, 78.73s/it]


Выучено батчей :    47; loss: 2.238
--------- Эпоха  2 / 3  ------------------


100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:03:09<00:00, 80.62s/it]

Выучено батчей :    47; loss: 2.149
 Обучение закончено!
 После обучения


    Валидационный набор: кросс-энтропия: 1.68, доля совпадений: 55.5%


Ещё раз

In [90]:
train_and_test(resnet50, train_dataset, valid_dataset, n_epochs=3)

 Обучаем сеть:  ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1,

100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:06:10<00:00, 84.48s/it]


Выучено батчей :    47; loss: 4.953
--------- Эпоха  1 / 3  ------------------


100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:08:23<00:00, 87.30s/it]


Выучено батчей :    47; loss: 2.146
--------- Эпоха  2 / 3  ------------------


100%|████████████████████████████████████████████████████████████████████████████████| 47/47 [1:03:45<00:00, 81.40s/it]

Выучено батчей :    47; loss: 2.107
 Обучение закончено!
 После обучения


    Валидационный набор: кросс-энтропия: 1.68, доля совпадений: 56.0%
